# Boletín Tema 6: Problemas de Optimización No Lineal

[![Open in Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/boiro9/OptimizacionIA/main?filepath=Boletin_ProgramacionNoLineal.ipynb)

🎯 En este jupyter se resolverán con Pyomo los problemas planteados en el boletín de ejercicios asociados al Tema 6 de Problemas de Optimización No Lineal. 

# 🚀  Instalación de paquetes:

Los requerimientos para poder ejecutar este jupyter son los siguientes:

In [27]:
# Instalar pyomo, amplpy, numpy y plotly
!pip install pyomo amplpy numpy plotly -q
# Instalar los solvers: HiGHS, CBC, Couenne, Bonmin, Ipopt, SCIP y GCG
!python -m amplpy.modules install coin highs scip gcg -q

<div style="border:1px solid #f0ad4e; padding:10px; border-radius:5px; background-color:#fcf8e3;">
<strong>⚠️ Advertencia: </strong>  <br>
Los nuevos paquetes que se utilizan son: <code>numpy</code> y <code>plotly</code>. Si ya has instalado previamente estos paquetes, y utilizas el mismo entorno de Python, <b>NO</b> es necesario que los vuelvas a instalar. 
</div>

# Problema 1

$$
\begin{array}{rl}
\max       & 3x+2y+z \\
\text{s.a.}& 2x^{2}+y^{2}+z \leq 10 \\
           & x+y+z\geq 1\\
					 & x\leq 0, z\geq 0
\end{array}
$$

In [2]:
import pyomo.environ as pe

modelo1 = pe.ConcreteModel()

# Variables
modelo1.x = pe.Var(within=pe.NonPositiveReals) 
#modelo1.x = pe.Var(bounds=(-float("Inf"),0)) # También se pueden especificar cotas con "bounds"
modelo1.y = pe.Var()
modelo1.z = pe.Var(within=pe.NonNegativeReals)

# Función objetivo
modelo1.obj = pe.Objective(expr=3*modelo1.x+2*modelo1.y+modelo1.z,sense=pe.maximize)

# Restricción 1:
modelo1.cons1 = pe.Constraint(expr=2*modelo1.x**2 + modelo1.y**2+modelo1.z <= 10)

# Restricción 2:
modelo1.cons2 = pe.Constraint(expr=modelo1.x + modelo1.y+modelo1.z >= 1)


Ahora especificamos el solver que queremos utilizar. Una vez lo definimos, para las siguiente resoluciones no es necesario definirlo de nuevo.

In [3]:
from amplpy import modules
solver_name = "ipopt"  # Opciones: "highs", "cbc", "couenne", "bonmin", "ipopt", "scip", "gcg".
solver_ipopt = pe.SolverFactory(
    solver_name+"nl",
    executable=modules.find(solver_name),
    solve_io="nl"
)

In [4]:
results = solver_ipopt.solve(modelo1)
print('*** Solucion *** :')
print('x:', modelo1.x())
print('y:', modelo1.y())
print('z:', modelo1.z())
print('Funcion objetivo: ',modelo1.obj())

*** Solucion *** :
x: 0.0
y: 0.9999999995823128
z: 9.000000098329602
Funcion objetivo:  11.000000097494228


In [5]:
modelo1.pprint()

3 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.0 :     0 : False : False : NonPositiveReals
    y : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :  None : 0.9999999995823128 :  None : False : False :  Reals
    z : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 9.000000098329602 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 3*x + 2*y + z

2 Constraint Declarations
    cons1 : Size=1, Index=None, Active=True
        Key  : Lower : Body              : Upper : Active
        None :  -Inf : 2*x**2 + y**2 + z :  10.0 :   True
    cons2 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :

A continuación calculamos los **multiplicadores de Lagrange** asociados a la solución obtenida:

In [6]:
modelo1.dual = pe.Suffix(direction=pe.Suffix.IMPORT)
results = solver_ipopt.solve(modelo1)

In [7]:
modelo1.dual.pprint()

dual : Direction=IMPORT, Datatype=FLOAT
    Key   : Value
    cons1 :      1.0000000005567407
    cons2 : -2.7835979663186923e-10


In [8]:
modelo1.dual[modelo1.cons1]

1.0000000005567407

In [9]:
modelo1.dual[modelo1.cons2]

-2.7835979663186923e-10

# Problema 2:

$$
\begin{array}{rl}
\min       & x+y\\
\text{s.a.}& x^{3}+y \leq 25 \\
           & x^{4}+2y \geq 16 \\
					 & x^{2}+3y \leq 20 \\
					 & y\geq 0
\end{array}
$$

Comenzamos representando gráficamemte la **región factible**:

In [10]:
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'iframe_connected'

fig = go.Figure()
xvar = np.linspace(-5,5,100) # -5<= x <= 5
cons1 = 25-xvar**3
cons2 = (16-xvar**4)/2
cons3 = (20-xvar**2)/3
ymin = min(cons1.min(),cons2.min(),cons3.min())
ymax = max(cons1.max(),cons2.max(),cons3.max())

fig.update_xaxes(
   range=[-5,5],  # sets the range of xaxis
    constrain="domain",                # meanwhile compresses the xaxis by decreasing its "domain"
)
fig.update_yaxes(
    range=[0,7],  # sets the range of xaxis
    constrain="domain",  # meanwhile compresses the xaxis by decreasing its "domain"
)

# Constraint 1
fig.add_trace(go.Scatter(x=xvar, y=cons1, fill=None, mode='lines', line_color='red', name="Cons1")) 
fig.add_trace(go.Scatter(x=[-5,5], y=[ymin,ymin], fill='tonexty',mode='lines', line_color='red', name="Cons1 Factible"))#fillcolor='lightblue', line=dict(color="white"))) 

# Constraint 2
fig.add_trace(go.Scatter(x=xvar, y=cons2, fill=None, mode='lines', line_color='blue', name="Cons2"))
fig.add_trace(go.Scatter(x=[-5,5], y=[ymax,ymax], fill='tonexty', mode='lines', line_color='blue', name="Cons2 Factible"))#, fillcolor='lightblue', line=dict(color="white"))) 

# Constraint 3
fig.add_trace(go.Scatter(x=xvar, y=cons3, fill=None, mode='lines', line_color='green', name="Cons3")) 
fig.add_trace(go.Scatter(x=[-5,5], y=[ymin,ymin], fill='tonexty', mode='lines', line_color='green', name="Cons3 Factible"))#, fillcolor='lightblue', line=dict(color="white")))

fig.show()

Se puede observar claramente que la región factible es **NO convexa**. A continuación resolvemos el problema empleando **pyomo**:

In [11]:
import pyomo.environ as pe

mod2 = pe.ConcreteModel()

# Variables
mod2.x = pe.Var() 
mod2.y = pe.Var() #Importante: no especificar cotas a la variable. 

# Función objetivo
mod2.obj = pe.Objective(expr=mod2.x+mod2.y,sense=pe.minimize)

# Restricción 1:
mod2.cons1 = pe.Constraint(expr=mod2.x**3 + mod2.y <= 25)

# Restricción 2:
mod2.cons2 = pe.Constraint(expr=mod2.x**4 + 2*mod2.y >= 16)

# Restricción 3:
mod2.cons3 = pe.Constraint(expr=mod2.x**2 + 3*mod2.y <= 20)

# Restricción 4:
mod2.cons4 = pe.Constraint(expr=mod2.y >= 0)

In [12]:
results2 = solver_ipopt.solve(mod2)
print('*** Solucion *** :')
print('x:', mod2.x())
print('y:', mod2.y())
print('Funcion objetivo: ',mod2.obj())

*** Solucion *** :
x: -4.472135977573311
y: -8.122380974988762e-09
Funcion objetivo:  -4.472135985695692


Comprobemos si la solución obtenida es un **punto KKT**. Recordemos que $\pmb{\bar{x}}$ es un punto **KKT** si cumple que:
\begin{eqnarray*}
\nabla f(\pmb{\bar{x}}) + \sum_{i\in I(\pmb{\bar{x}})}u_{i}\nabla g_{i}(\pmb{\bar{x}})& = &\pmb{0} \\
u_{i} & \geq & 0, \, \forall i\in I(\pmb{\bar{x}}) \\
\end{eqnarray*}
donde $I(\pmb{\bar{x}})$ es el índice de las restricciones saturadas en $\pmb{\bar{x}}$.

Comprobemos cuáles son las restricciones saturadas en $\pmb{\bar{x}}=(-4.47,0)$. Para ello, hacemos un `display` del objeto modelo ya resuelto:

In [13]:
# Miramos que restricciones están saturadas:
mod2.display()

Model unknown

  Variables:
    x : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :  None : -4.472135977573311 :  None : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value                  : Upper : Fixed : Stale : Domain
        None :  None : -8.122380974988762e-09 :  None : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -4.472135985695692

  Constraints:
    cons1 : Size=1
        Key  : Lower : Body               : Upper
        None :  None : -89.44272046253788 :  25.0
    cons2 : Size=1
        Key  : Lower : Body             : Upper
        None :  16.0 : 400.000008059979 :  None
    cons3 : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 20.00000017753845 :  20.0
    cons4 : Size=1
        Key  : Lower : Body                   : Upper
        None :   0.0 : -8.12238097498876

Vemos que la única restricción saturada es la tercera (`cons3`) y la cuarta (`cons4`). Por lo tanto solo nos tenemos que fijar en: 
- $g_{3}(x,y)=x^{2}+3y\leq 20$
- $g_{4}(x,y)=-y\leq 0$. (NOTA: La convertimos en restricción de $\leq$)

Calcular los gradientes de la función objetivo y las restricciones (solo sería necesario para $g_3$ y $g_{4}$) en dicho punto:
\begin{eqnarray*}
\nabla f(x,y)     & = & (1,1) \Rightarrow \nabla f(-4.47,0) = (1,1)\\
\nabla g_{1}(x,y) & = & (3x,1) \Rightarrow \nabla g_{1}(-4.47,0) = (-13.42,1)\\
\nabla g_{2}(x,y) & = & (4x,2) \Rightarrow \nabla g_{2}(-4.47,0) = (-17.89,2)\\
\nabla g_{3}(x,y) & = & (2x,3) \Rightarrow \nabla g_{3}(-4.47,0) = (-8.94,3)\\
\nabla g_{4}(x,y) & = & (0,-1) \Rightarrow \nabla g_{3}(-4.47,0) = (0,-1)\\
\end{eqnarray*}

Llamemos $u_{3}$ y $u_{4}$ al multiplicador asociados a la restricción $g_{3}$ y $g_{4}$, respectivamente. Entonces:
$$
\nabla f(\pmb{\bar{x}}) + \sum_{i\in I(\pmb{\bar{x}})}u_{i}\nabla g_{i}(\pmb{\bar{x}}) = \left( \begin{array}{c} 1 \\ 1 \end{array} \right) + u_{3}\left( \begin{array}{c} −8.94 \\ 3 \end{array} \right) + u_{4}\left( \begin{array}{c} 0 \\ -1 \end{array} \right) = \pmb{0}
$$

De la primera ecuación obtenemos que:
$$
u_{3}=\dfrac{1}{8.94}=0.11
$$
De la segunda ecuación se obtiene:
$$
u_{4}=1+3\cdot \dfrac{1}{8.94} = 1.335
$$
Por lo tanto, existe $u_{3}=0.11\geq 0$ y $u_{4}=1.335\geq 0$ verificando dichas condiciones, por lo que podemos afirmar que $\pmb{\bar{x}}=(-4.47,0)$ es un punto **KKT** del problema.

A continuación calculamos los multiplicadores de Lagrange pidiéndole a Pyomo que nos diga los valores duales asociados a las restricciones. Para ello, debemos de especcificar al solver que nos pase dicha información y resolver de nuevo.

In [14]:
# Calculamos los duales asociados a las restricciones:
mod2.dual = pe.Suffix(direction=pe.Suffix.IMPORT)
results = solver_ipopt.solve(mod2)
mod2.dual.pprint()

dual : Direction=IMPORT, Datatype=FLOAT
    Key   : Value
    cons1 : -2.185162711998076e-11
    cons2 :  6.508432959926355e-12
    cons3 :   -0.11180339871779163
    cons4 :     1.3354101961622098


Notemos que el dual asociado a la restricción $g_{3}$ y $g_{4}$ coinciden con los multiplicadores que hemos obtenido analíticamente (el de $g_3$ con signo contrario).

Notemos que aunque $\pmb{\bar{x}}=(-4.47,0)$ es un punto **KKT** del problema, no podemos afirmar que se trata del óptimo global (ni siquiera de un óptimo local) puesto que el problema es no convexo (la región factible es no convexa) y por lo tanto las condiciones de $KKT$ no son suficientes. 